In [262]:
import torch
import numpy as np
import scipy
from scipy.linalg import expm
import sinkhorn as sk

In [263]:
dim = 5

In [264]:
sk = sk.SinkhornKnopp()
seed = np.random.random((dim,dim))
seed = sk.fit(seed)

In [265]:
def generate_cities(n):
    """
    Generate an n*2 array of random number between 0 and d

    Returns
    -------
    array : cities positions

    """
    array = np.random.rand(n, 2) * 100
    return array

def generate_cost_array(n,cities):
    """
    Generate an n*n array of distance cost (2-norm)

    Returns
    -------
    array : cost (distance)

    """
    array = np.zeros((n, n))
    for var_1 in range(0, n):
        for var_2 in range(0, n):
            vector = cities[var_1] - cities[var_2]
            array[var_1, var_2] = np.linalg.norm(vector)
            if var_1 == var_2:
                array[var_1, var_2] = 0

    return array

In [266]:
def generate_default_solution(n):
    """
    Set the start array

    """
    array = np.zeros((n, n))
    array[n - 1, 0] = 1
    for var in range(0, n - 1):
        array[0 + var, 1 + var] = 1
    return array

def shuffle_perm(P):
    perm = np.random.permutation(dim)

    # Create an empty matrix
    res = np.zeros((dim,dim))

    # Assign 1 to the permuted indices
    for i in range(dim):
        res[i, perm[i]] = 1
    
    return res

In [267]:
A = generate_default_solution(dim)
M = shuffle_perm(dim)
# P = generate_default_solution(dim) + 0.01
P = (M+0.1)
# P = seed
Cts = generate_cities(dim)
C = generate_cost_array(dim,Cts)
L1 = np.array([100])
L2 = np.array([100])

In [268]:
np.round(P)

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.]])

In [269]:
M

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.]])

In [270]:
def calculate_cost(p,l1):
    cost = C.T @ p.T @ A @ p
    constraint_1 = np.trace(p.T @ p - np.identity(dim))
    return np.trace(cost) + (l1 * constraint_1)

In [271]:
def grad_P_his(p):
    delta = 0.000000001
    Gradient = P.copy()
    for i in range(len(P)):
        for j in range(len(P[i])):
            tmp1 = P.copy()
            tmp1[i][j] = tmp1[i][j]+delta
            tmp2 = P.copy()
            tmp2[i][j] = tmp2[i][j]-delta
            Gradient[i][j] = (calculate_cost(tmp1,0)-calculate_cost(tmp2,0))/(delta*2)

    return (Gradient)

In [272]:
def grad_P(p):
    return A @ P @ C.T + A.T @ P.T @ C

In [273]:
def procrustes(p):
    print(np.shape(p))
    u,s,w = np.linalg.svd(p)
    res_r = np.linalg.matrix_rank(s)
    first_ru = u[0:res_r,:]
    first_rw = w[:,0:res_r]
    res = u.T @ w
    print(np.shape(res))
    return res

In [274]:
def projection_DP(Y,X):
    alpha = np.linalg.pinv(1 - X @ X.T) @ (Y - X @ Y.T) @ np.ones(dim)
    beta = Y.T @ np.ones(dim) - X.T @ alpha
    res = Y - np.multiply((alpha @ alpha.T + np.ones(dim) @ beta),X)

    return res

In [275]:
def grad_DP(X):
    element = np.multiply(grad_P(X),X)
    return projection_DP(element,X)

In [276]:
def retraction_DP(p,xX):
    res = np.multiply(p,expm(np.divide(xX,p)))
    return res

In [277]:
def nearest_bistochastic(p):
        I = np.identity(dim)
        J = np.ones((dim,dim)) / dim
        W = I - J
        B = W @ p @ W + J
        return B

In [278]:
def calc_stepArmijo(p ,g ,tau, c):
    step = 1e-5

    t = c * 1
    j = 0
    max = 100
    while ((calculate_cost(p,L1) - calculate_cost(p+step*g,L1)) < step * t) and (j < max):
        step = tau*step
        j = j+1
    
    print(step)
    return step

In [279]:
cost_history = []
cost_history.append(calculate_cost(M,0))
P_history = []
P_history.append(M)
iter = 0
iter_max = 20

while iter < iter_max:
    iter = iter + 1
    xiX = -grad_DP(P)
    # step = calc_stepArmijo(P,xiX,0.9,0.9)
    step = 0.0000000000001
    print(retraction_DP(P,step * xiX))
    P = sk.fit(abs(retraction_DP(P,step * xiX)))
    print(sum(P))
    # P = procrustes(P)
    gL1 = np.trace(P.T @ P - np.identity(dim))
    L1 = L1 + 0.01 * gL1
    # gL2 = 1/3 * np.trace(P.T @ (P - (np.multiply(P,P))))
    # L2 = L2 + step * gL2
    cost_history.append(calculate_cost(P,0))
    P_history.append(P)

[[1.00000000e-01 1.30927757e-10 1.29881635e-10 1.31847599e-10
  1.43764699e-09]
 [1.30937481e-10 1.00000000e-01 1.30597166e-10 1.44826400e-09
  1.31054445e-10]
 [1.30788257e-10 1.44030196e-09 1.00000000e-01 1.30949369e-10
  1.30711620e-10]
 [1.42944805e-09 1.31202131e-10 1.31416125e-10 1.00000000e-01
  1.31550053e-10]
 [1.30389791e-10 1.31671326e-10 1.44041622e-09 1.31095457e-10
  1.00000000e-01]]
[1. 1. 1. 1. 1.]
[[9.99999982e-01 8.57822710e-20 7.81028263e-20 7.89524677e-20
  9.41928785e-19]
 [7.65502535e-20 9.99999982e-01 7.63512942e-20 8.46086328e-19
  7.39690466e-20]
 [7.84146902e-20 9.43839546e-19 9.99999982e-01 8.58120005e-20
  7.82566828e-20]
 [7.15851449e-19 7.98060965e-20 8.51915558e-20 9.99999982e-01
  8.52783762e-20]
 [8.78866718e-20 7.64787774e-20 7.12730953e-19 8.83623122e-20
  9.99999982e-01]]
[1. 1. 1. 1. 1.]
[[1.00000000e+00 5.62035014e-30 4.69662382e-30 4.72780125e-30
  6.17140293e-29]
 [4.47537371e-30 1.00000000e+00 4.46374191e-30 4.94289791e-29
  4.17492143e-30]
 [4.

In [280]:
P.T @ P

array([[1.00000000e+000, 4.73336647e-203, 1.61147835e-203,
        1.05476973e-203, 5.38702040e-202],
       [4.73336647e-203, 1.00000000e+000, 4.73567883e-202,
        6.35335776e-203, 6.82878901e-204],
       [1.61147835e-203, 4.73567883e-202, 1.00000000e+000,
        7.74363271e-203, 9.89600795e-204],
       [1.05476973e-203, 6.35335776e-203, 7.74363271e-203,
        1.00000000e+000, 1.07719225e-202],
       [5.38702040e-202, 6.82878901e-204, 9.89600795e-204,
        1.07719225e-202, 1.00000000e+000]])

In [281]:
print(retraction_DP(P,step * xiX))
print(step*xiX)


[[1.12139336e+001 4.60526018e-202 8.95531488e-203 8.39306124e-203
  5.05678747e-201]
 [5.28574708e-203 1.12139336e+001 5.27200906e-203 5.76622598e-202
  2.71075832e-203]
 [8.52286649e-203 5.08362280e-201 1.12139336e+001 4.62192800e-202
  8.28941283e-203]
 [3.04702833e-203 1.12466023e-202 3.77685794e-202 1.12139336e+001
  3.78070701e-202]
 [7.86004005e-202 4.69578319e-203 2.44385095e-203 7.90257839e-202
  1.12139336e+001]]
[[5.44785554e-011 4.24602360e-203 8.25674921e-204 7.73835457e-204
  4.66232919e-202]
 [4.87342864e-204 5.81709416e-011 4.86076227e-204 5.31642744e-203
  2.49930370e-204]
 [7.85803427e-204 4.68707121e-202 5.66362148e-011 4.26139124e-203
  7.64279132e-204]
 [2.80934273e-204 1.03693031e-203 3.48224147e-203 5.56791070e-011
  3.48579029e-203]
 [7.24691207e-203 4.32948530e-204 2.25321663e-204 7.28613218e-203
  5.62770754e-011]]


In [282]:
print("Original Solution Result: \t",calculate_cost(M,0))
print("Optimised Solution Result: \t",calculate_cost(np.round(P),0))
print("Identity Solution Result: \t",calculate_cost(np.identity(dim),0))

Original Solution Result: 	 352.0837251271439
Optimised Solution Result: 	 351.55236769872886
Identity Solution Result: 	 351.55236769872886


In [283]:
print(np.round(P))
print(P)
print(cost_history)

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
[[1.00000000e+000 4.24602361e-203 8.25674921e-204 7.73835457e-204
  4.66232919e-202]
 [4.87342864e-204 1.00000000e+000 4.86076227e-204 5.31642744e-203
  2.49930370e-204]
 [7.85803427e-204 4.68707121e-202 1.00000000e+000 4.26139124e-203
  7.64279132e-204]
 [2.80934273e-204 1.03693031e-203 3.48224147e-203 1.00000000e+000
  3.48579029e-203]
 [7.24691207e-203 4.32948530e-204 2.25321663e-204 7.28613218e-203
  1.00000000e+000]]
[352.0837251271439, 351.5523654746044, 351.55236769872886, 351.55236769872886, 351.55236769872886, 351.55236769872886, 351.55236769872886, 351.55236769872886, 351.55236769872886, 351.55236769872886, 351.55236769872886, 351.55236769872886, 351.55236769872886, 351.55236769872886, 351.55236769872886, 351.55236769872886, 351.55236769872886, 351.55236769872886, 351.55236769872886, 351.55236769872886, 351.55236769872886]
